In [ ]:
# Install required packages
# !pip install langgraph langchain-google-genai langchain-community duckduckgo-search

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY") 
print(f"API Key loaded: {API_KEY[:10]}..." if API_KEY else "No API Key found")

In [ ]:
from typing import TypedDict, Annotated, Sequence, Literal
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
import json

In [ ]:
# Define the state for our research team
class ResearchState(TypedDict):
    """The state of the research team."""
    messages: Annotated[Sequence[BaseMessage], add_messages]
    user_query: str  # Original user question
    search_results: str  # Results from searcher
    draft: str  # Draft from writer
    critique: str  # Feedback from critic
    final_answer: str  # Final compiled answer
    next_agent: str  # Which agent to call next
    iteration_count: int  # Track iterations to prevent infinite loops

In [ ]:
# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.7,
    google_api_key=API_KEY
)

print("LLM initialized successfully")

In [ ]:
# Supervisor Node - Orchestrates the research team
def supervisor_node(state: ResearchState) -> ResearchState:
    """
    The supervisor reviews the current state and decides which agent to call next.
    """
    user_query = state.get("user_query", "")
    search_results = state.get("search_results", "")
    draft = state.get("draft", "")
    critique = state.get("critique", "")
    iteration = state.get("iteration_count", 0)
    
    # Create supervisor prompt
    supervisor_prompt = f"""You are a research supervisor coordinating a team of agents.

User Query: {user_query}

Current State:
- Search Results: {"Available" if search_results else "Not yet gathered"}
- Draft: {"Available" if draft else "Not yet written"}
- Critique: {"Available" if critique else "Not yet reviewed"}
- Iteration: {iteration}

Your team consists of:
1. Searcher - Finds relevant information and data
2. Writer - Synthesizes information into a coherent draft
3. Critic - Reviews drafts for accuracy and completeness

Based on the current state, decide the next step:
- If no search results exist, call "Searcher"
- If search results exist but no draft, call "Writer"
- If draft exists but no critique, call "Critic"
- If critique suggests improvements and iteration < 2, call "Writer" for revision
- If work is complete or iteration >= 2, call "Finish"

Respond with ONLY ONE WORD: Searcher, Writer, Critic, or Finish"""
    
    response = llm.invoke(supervisor_prompt)
    decision = response.content.strip()
    
    print(f"\n🎯 SUPERVISOR DECISION: {decision}")
    
    return {
        "next_agent": decision,
        "iteration_count": iteration
    }

In [ ]:
# Searcher Agent Node
def searcher_agent(state: ResearchState) -> ResearchState:
    """
    Searches for relevant information using web search (simulated).
    """
    user_query = state.get("user_query", "")
    
    print("\n🔍 SEARCHER AGENT: Gathering information...")
    
    # Create search prompt for LLM to simulate search results
    search_prompt = f"""You are a research searcher agent. Your job is to find and summarize 
relevant information about the following query:

Query: {user_query}

Provide a comprehensive summary of key facts, statistics, pros, cons, and relevant 
information that would help answer this query. Format your response as if you've 
gathered information from multiple authoritative sources.

Include:
- Key facts and statistics
- Different perspectives
- Recent developments
- Expert opinions"""
    
    response = llm.invoke(search_prompt)
    search_results = response.content
    
    print(f"✅ Search completed: {len(search_results)} characters of data gathered")
    
    return {
        "search_results": search_results,
        "messages": [AIMessage(content=f"Search completed for: {user_query}")]
    }

In [ ]:
# Writer Agent Node
def writer_agent(state: ResearchState) -> ResearchState:
    """
    Synthesizes search results into a coherent draft answer.
    """
    user_query = state.get("user_query", "")
    search_results = state.get("search_results", "")
    critique = state.get("critique", "")
    previous_draft = state.get("draft", "")
    
    print("\n✍️ WRITER AGENT: Crafting response...")
    
    # Build the revision section if there's a critique
    revision_section = ""
    if critique:
        revision_section = f"\nPrevious Draft:\n{previous_draft}\n\nCritique to Address:\n{critique}\n"
    
    # Build the critique instruction
    critique_instruction = "- Addresses the critique points" if critique else ""
    
    # Create writer prompt
    writer_prompt = f"""You are a research writer agent. Your job is to synthesize 
information into a clear, well-structured answer.

User Query: {user_query}

Search Results:
{search_results}
{revision_section}
Write a comprehensive, well-structured answer that:
- Directly addresses the user's question
- Presents information clearly and logically
- Includes relevant facts and perspectives
- Is balanced and objective
{critique_instruction}

Provide your draft answer:"""
    
    response = llm.invoke(writer_prompt)
    draft = response.content
    
    print(f"✅ Draft completed: {len(draft)} characters")
    
    return {
        "draft": draft,
        "messages": [AIMessage(content="Draft answer created")],
        "iteration_count": state.get("iteration_count", 0) + 1
    }

In [ ]:
# Critic Agent Node
def critic_agent(state: ResearchState) -> ResearchState:
    """
    Reviews the draft for accuracy, completeness, and clarity.
    """
    user_query = state.get("user_query", "")
    draft = state.get("draft", "")
    search_results = state.get("search_results", "")
    
    print("\n🔎 CRITIC AGENT: Reviewing draft...")
    
    # Create critic prompt
    critic_prompt = f"""You are a research critic agent. Your job is to review drafts 
for accuracy, completeness, and clarity.

User Query: {user_query}

Draft Answer:
{draft}

Available Research:
{search_results[:500]}...

Evaluate the draft and provide:
1. What's good about it
2. What's missing or could be improved
3. Any inaccuracies or unclear points
4. Overall assessment: APPROVED or NEEDS_REVISION

Be constructive and specific in your feedback."""
    
    response = llm.invoke(critic_prompt)
    critique = response.content
    
    print(f"✅ Critique completed")
    
    return {
        "critique": critique,
        "messages": [AIMessage(content="Draft reviewed by critic")]
    }

In [ ]:
# Router function for conditional edges
def route_after_supervisor(state: ResearchState) -> Literal["searcher", "writer", "critic", "finish"]:
    """
    Routes to the appropriate agent based on supervisor's decision.
    """
    next_agent = state.get("next_agent", "").lower()
    
    if "searcher" in next_agent:
        return "searcher"
    elif "writer" in next_agent:
        return "writer"
    elif "critic" in next_agent:
        return "critic"
    else:
        return "finish"

In [ ]:
# Finalize node - prepares the final answer
def finalize_answer(state: ResearchState) -> ResearchState:
    """
    Prepares the final answer for the user.
    """
    draft = state.get("draft", "")
    critique = state.get("critique", "")
    
    print("\n✨ FINALIZING ANSWER...")
    
    final_answer = f"""# Research Team Final Answer

{draft}

---
Quality Review: {"Approved by critic" if critique else "Completed"}
"""
    
    return {
        "final_answer": final_answer,
        "messages": [AIMessage(content="Research complete!")]
    }

In [ ]:
# Build the research team graph
workflow = StateGraph(ResearchState)

# Add all nodes
workflow.add_node("supervisor", supervisor_node)
workflow.add_node("searcher", searcher_agent)
workflow.add_node("writer", writer_agent)
workflow.add_node("critic", critic_agent)
workflow.add_node("finalize", finalize_answer)

# Set entry point
workflow.add_edge(START, "supervisor")

# Add conditional edges from supervisor to agents
workflow.add_conditional_edges(
    "supervisor",
    route_after_supervisor,
    {
        "searcher": "searcher",
        "writer": "writer",
        "critic": "critic",
        "finish": "finalize"
    }
)

# All worker agents route back to supervisor
workflow.add_edge("searcher", "supervisor")
workflow.add_edge("writer", "supervisor")
workflow.add_edge("critic", "supervisor")

# Finalize goes to END
workflow.add_edge("finalize", END)

# Compile the graph
research_graph = workflow.compile()

print("✅ Research Team Graph compiled successfully!")

In [ ]:
# Visualize the graph
try:
    from IPython.display import Image, display
    display(Image(research_graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not visualize graph: {e}")

## How the Research Team Works

### Architecture
1. **Supervisor Node**: Orchestrates the workflow by deciding which agent to call next
2. **Searcher Agent**: Gathers relevant information and data
3. **Writer Agent**: Synthesizes information into a coherent draft
4. **Critic Agent**: Reviews drafts for quality and completeness
5. **Finalize Node**: Prepares the final answer

### Workflow
1. User submits a complex research question
2. Supervisor analyzes the state and routes to Searcher
3. Searcher gathers information → back to Supervisor
4. Supervisor routes to Writer
5. Writer creates draft → back to Supervisor
6. Supervisor routes to Critic
7. Critic reviews draft → back to Supervisor
8. Supervisor decides: revise (back to Writer) or finish
9. Finalize prepares the final answer

### Key Features
- **Conditional Routing**: Supervisor uses LLM to make intelligent routing decisions
- **Iterative Refinement**: Can loop back for revisions based on critique
- **State Management**: Tracks all intermediate results and progress
- **Loop Prevention**: Iteration counter prevents infinite loops

In [ ]:
# Test 1: Complex research question
print("="*70)
print("TEST 1: Nuclear Fusion vs Solar Power")
print("="*70)

initial_state = {
    "messages": [],
    "user_query": "What are the pros and cons of nuclear fusion vs. solar power for base-load energy by 2050?",
    "search_results": "",
    "draft": "",
    "critique": "",
    "final_answer": "",
    "next_agent": "",
    "iteration_count": 0
}

result = research_graph.invoke(initial_state)

print("\n" + "="*70)
print("FINAL ANSWER:")
print("="*70)
print(result["final_answer"])

In [ ]:
# Test 2: Another complex question
print("\n" + "="*70)
print("TEST 2: AI Impact on Employment")
print("="*70)

initial_state_2 = {
    "messages": [],
    "user_query": "How will artificial intelligence impact employment in the healthcare sector over the next decade?",
    "search_results": "",
    "draft": "",
    "critique": "",
    "final_answer": "",
    "next_agent": "",
    "iteration_count": 0
}

result_2 = research_graph.invoke(initial_state_2)

print("\n" + "="*70)
print("FINAL ANSWER:")
print("="*70)
print(result_2["final_answer"])

In [ ]:
# Interactive: Ask your own research question
def ask_research_team(question: str):
    """
    Helper function to ask the research team a question.
    """
    print("\n" + "="*70)
    print(f"RESEARCH QUESTION: {question}")
    print("="*70)
    
    state = {
        "messages": [],
        "user_query": question,
        "search_results": "",
        "draft": "",
        "critique": "",
        "final_answer": "",
        "next_agent": "",
        "iteration_count": 0
    }
    
    result = research_graph.invoke(state)
    
    print("\n" + "="*70)
    print("FINAL ANSWER:")
    print("="*70)
    print(result["final_answer"])
    
    return result

# Example usage:
# ask_research_team("What are the environmental impacts of cryptocurrency mining?")